In [0]:
'''
  _____  _    _ _   _   _______ _    _ _____  _____    _____ ______ _      _        ______ _____ _____   _____ _______ 
 |  __ \| |  | | \ | | |__   __| |  | |_   _|/ ____|  / ____|  ____| |    | |      |  ____|_   _|  __ \ / ____|__   __|
 | |__) | |  | |  \| |    | |  | |__| | | | | (___   | |    | |__  | |    | |      | |__    | | | |__) | (___    | |   
 |  _  /| |  | | . ` |    | |  |  __  | | |  \___ \  | |    |  __| | |    | |      |  __|   | | |  _  / \___ \   | |   
 | | \ \| |__| | |\  |    | |  | |  | |_| |_ ____) | | |____| |____| |____| |____  | |     _| |_| | \ \ ____) |  | |   
 |_|  \_\\____/|_| \_|    |_|  |_|  |_|_____|_____/   \_____|______|______|______| |_|    |_____|_|  \_\_____/   |_|  
'''
# Disturbance profiles were uploaded to Github so markers can access them through this notebook. Let's get them.
!rm -rf CHBE356-data
!git clone https://github.com/TimHillmer/CHBE356-data/


# obtain a numerically ordered list of the paths to the disturbance profiles
import glob
filenames = glob.glob('CHBE356-data/Disturbance_profiles/*.csv')
filenames.sort()

# obtain an array containing all disturbance profiles (data in [profile#, timevalue] format)
import numpy as np
import pandas as pd
df = pd.concat([pd.read_csv(files).transpose() for files in filenames], ignore_index = True)
# each disturbance profile is a *row*

# define Yankai's 100 point interpolation
from scipy.interpolate import interp1d
Ti_data = df.values
time_data = np.arange(len(Ti_data[0]))  # assume Ti' data is given at each second
f = interp1d(time_data, Ti_data)
points_per_second = 100
times = np.linspace(time_data[0], time_data[-1], len(time_data)*points_per_second)
TiPrimes = f(times)

# Any remaining install/imports here
! pip install git+https://github.com/python-control/python-control@601b58152080d89575cc677474ec7714e1a34ee2
import control
import matplotlib.pyplot as plt
import random


def fasterplan(KC, TauI, TauD, TauC, s, times, Gd, Gp):  # streamlined version to save precious milliseconds each run
    Gc = KC*(1 + 1/(TauI*s) + TauD*s/(TauC*s+1))  # Controller transfer function
    Gtotal = Gd / (1 + Gp*Gc)  # no term for (temperature deviation) setpoint because it is always zero
    Perf = np.zeros(len(TiPrimes))  # array to store Perf for each disturbance profile
    for i in range(len(TiPrimes)):  # main loop for calculation and plotting
        # T' response including controller
        t, Tprime, x = control.forced_response(Gtotal, times, TiPrimes[i])

        # controller response to error (=input)
        t, Qprime, x = control.forced_response(Gc, times, -Tprime)

        Perf[i] = (sum(np.absolute(Tprime)) + 0.2*sum(np.absolute(Qprime)))/points_per_second  
        # performance metric for each profile
    return np.average(Perf)

  
def lograndom(lo, hi):  # samples from log(X), where X in uniformly distributed within the provided bounds
    return(lo*10**(random.uniform(0, np.log10(hi/lo))))
         
         
# initial information for use in "fasterplan"
s = control.tf([1,0],[0,1])  # Define the s variable
Gd = (s+1) / (s**2+s+1)  # T' / Ti' (disturbance) transfer function
Gp = 1 / (s**2+s+1)  # T' / Q' (process) transfer function

In [0]:
Ntests = 500
Nhappy = int(Ntests/10)

counter = 0
happycounter = 0

data = np.zeros((Ntests, 5))
happydata = np.zeros((Nhappy, 5))
from google.colab import files

for i in range(Ntests):
    Kc = lograndom(1e3, 1e4)
    Ti = lograndom(1e-4, 1e-3)
    Td = lograndom(0.05,1.5)
    Tc = lograndom(5e-4, 0.1)

    Perf = fasterplan(Kc, Ti, Td, Tc, s, times, Gd, Gp)
    if Perf <= 6.95:
        happydata[happycounter] = np.array([Perf, Kc, Ti, Td, Tc])
        happycounter += 1
        print('happy', Perf, Kc, Ti, Td, Tc)

    data[counter] = np.array([Perf, Kc, Ti, Td, Tc]) 
    counter += 1
    
    if happycounter == Nhappy:
        break

In [0]:
from google.colab import files  # this notebook was run with Colaboratory, which downloads files using this library
np.savetxt("Data.csv", data, delimiter=",")  
np.savetxt("HappyData.csv", happydata, delimiter=",") 
files.download('HappyData.csv')

In [0]:
fasterplan(5688.814947611792, 0.0002280222206125743, 0.4941522520819933, 0.006245557968152839, s, times, Gd, Gp)
# happy 6.789263931270320 8157.95407778562 0.00014069840043239406 0.13349633544031736 0.0010902891735579206
# happy 6.769239197968654 6780.9431099381745 0.0008394599924216926 0.6232736552360005 0.01064341130569476
# happy 6.661657297636582 9535.386683958759 0.00010033885457162226 0.819218758695484 0.004582526747423241
# happy 6.604902873671916 8763.00366411034 0.00021935638604842907 0.17331925291467143 0.0034497938673252715
# happy 6.603490145150944 5688.814947611792 0.0002280222206125743 0.4941522520819933 0.006245557968152839